In [1]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown
url = "https://www.janestreet.com/puzzles/current-puzzle/"
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
text_blocks = [text for text in soup.body.stripped_strings]
puzzle_text = " ".join(text_blocks[14:55])
full_text = "## Robot Road Trip - July 2025\n\n" + puzzle_text
Markdown(full_text)

## Robot Road Trip - July 2025

Robot cars have a top speed (which they prefer to maintain at all times while driving)
that’s a real number randomly drawn uniformly between 1 and 2 miles per minute. A two-lane
highway for robot cars has a fast lane (with minimum speed a ) and a slow lane (with
maximum speed a ). When a faster car overtakes a slower car in the same lane, the slower
car is required to decelerate to either change lanes (if both cars start in the fast lane)
or stop on the shoulder (if both cars start in the slow lane). Robot cars decelerate and
accelerate at a constant rate of 1 mile per minute per minute, timed so the faster,
overtaking car doesn’t have to change speed at all, and passing happens
instantaneously. If cars rarely meet (so you never have to consider a car meeting more
than one other car on its trip, see Mathematical clarification below), and you want to
minimize the miles not driven due to passing, what should a be set to, in miles per
minute? Give your answer to 10 decimal places . Example car interactions: suppose a is set to 1.2 miles per minute. If a car with top
speed 1.8 overtakes a car with top speed 1.1, neither has to slow down because they are in
different lanes. If instead the car with top speed 1.8 overtakes one with top speed 1.7,
the slower car computes the optimal time to start decelerating for 30 seconds (to reach
1.2 miles per minute to switch to the other lane) so the faster car instantly passes and
the slower car can immediately start accelerating for another 30 seconds to return to 1.7
miles per minute. This pass cost 0.25 miles (how far behind where the slower car would be
if it continued at 1.7 miles per minute). If a car with top speed 1.1 overtakes one with top speed 1.0 in the slow lane, the slower
(slowest!) car must decelerate for a full minute all the way to 0 to allow the pass, and
then accelerate for a full minute to reestablish its speed, losing exactly 1 mile of
distance. Assume all car trips are of constant length N , starting at arbitrary points and times
along an infinitely long highway. This is made more mathematically precise below. Mathematical clarification : Say car trips arrive at a rate of z car trip beginnings
per mile per minute, uniformly across the infinite highway (cars enter and exit their
trips at their preferred speed due to on/off ramps), and car trips have a constant length
of N miles. Define f ( z , N ) to be the value of a that minimizes the expected lost
distance per car trip due to passing. Find: the limit of [the limit of f ( z , N ) as z -> 0+] as N -> infinity.

Speed Distribution: Each car's constant speed is modelled as a uniform distribution S ~ Uniform[1, 2]

Cars go in an assigned lane based on their speed:
<br>-Slow Lane: S &le; a
<br>-Fast Lane: S &ge; a

Assumptions:
We assume each car overtakes at most one other car

In [ ]:
# PDF of S
import sympy as sp
s = sp.symbols('s')
f = sp.Piecewise((1, (s >= 1) & (s <= 2)), (0, True))

#print(sp.integrate(f, (s, 1, 2)) == 1)

Distance Loss Per Overtake:

Slow Lane (S &le; a):
-Slower car decelerates to 0 then re-accelerates to <em>y</em> at unit rate:<code>L<sub>slow</sub>(y) = y<sup>2</sup></code><br>
Fast Lane (S &ge; a):
-Slower car decelerates to 0 then re-accelerates to <em>y</em> at unit rate:<code>L<sub>fast</sub>(y) = (y-a)<sup>2</sup></code><br>
Weight each overtake by relative speed (x - y) and integrate over 1 &le; y &lt; x &le; 2:
<br>
<code>
I<sub>slow</sub>(a) = &int;<sub>x=1</sub><sup>a</sup> &int;<sub>y=1</sub><sup>x</sup> (x &minus; y) y<sup>2</sup> dy dx,<br>
I<sub>fast</sub>(a) = &int;<sub>x=a</sub><sup>2</sup> &int;<sub>y=a</sub><sup>x</sup> (x &minus; y)(y &minus; a)<sup>2</sup> dy dx,<br>
g(a) = I<sub>slow</sub>(a) + I<sub>fast</sub>(a).
</code>


In [ ]:
# Symbolic integration to derive g(a)
a, x, y = sp.symbols('a x y')
I_slow = sp.integrate(sp.integrate((x-y)*y**2, (y, 1, x)), (x, 1, a))
I_fast = sp.integrate(sp.integrate((x-y)*(y-a)**2, (y, a, x)), (x, a, 2))
g = sp.simplify(I_slow + I_fast)
print('g(a) =', g)

g(a) = a**4/6 - 2*a**3/3 + 7*a**2/6 - 13*a/12 + 13/30


Finding the value of a that minimises the expected loss function

Compute the derivative g'(a) and find roots in [1, 2]
Confirm that the second derivative is positive at optimum

In [ ]:
dg = sp.diff(g, a)
crit = sp.nroots(dg)
real_crit = [r for r in crit if abs(sp.im(r)) < 1e-8]
float_roots = [float(sp.re(r)) for r in real_crit]
opt = [r for r in float_roots if 1 < r < 2][0]
print('Critical a=', opt)
# second derivative
d2g = sp.diff(dg, a)
print('Minimum Value?', d2g.subs(a, opt).evalf() > 0)

Critical a= 1.2266988257582019
Minimum Value? True


Numeric Check

valuate g(a) near the candidates optimum to verify it is minimal

In [ ]:
import numpy as np
values = [1.2, 1.2266988257582019, 1.25]
for a in values:
    print(f"a = {a}, g(a) = {a**4/6 - 2*a**3/3 + 7*a**2/6 - 13*a/12 + 13/30}")

a = 1.2, g(a) = 0.006933333333333236
a = 1.2266988257582019, g(a) = 0.0067806865234583125
a = 1.25, g(a) = 0.0069010416666668295


Final Answer: 

a = 1.2266988258